In [ ]:
using Plots, Distributions, Random, StatsBase
using Interact

p = plot()
@manipulate for input_mean in range(0.0, 0.5, 100), input_std in range(0.0, 0.5, 100)
    # Define the ELU function
    function elu(x, α=1.0)
        return x >= 0 ? x : α * (exp(x) - 1)
    end

    # Define the transformed ELU function from your example
    transformed_elu(x) = elu.((x .- 2.45) * 6, 1.0) .+ 1.0

    function inverse_transformed_elu(y)
        w = y - 1.0
        if w >= 0
            return w / 6 + 2.45
        elseif y <= 0
            return NaN
        else
            # 0 < y < 1.0 (since the range of transformed_elu is (0, ∞))
            return log(y) / 6 + 2.45
        end
    end

    # Set up the plot
    x = range(1.0, 3.0, 1001)
    y = transformed_elu(x)

    # Create the main plot
    p = plot(x, y, linewidth=3, color=:blue, label="Nonlinear Function",
        ylabel="Qi [gB]", xlabel="-a/LTᵢ",
        title="Non-linear Error Propagation",
        grid=true, gridwidth=1, gridcolor=:gray, gridalpha=0.3)

    # Generate random samples from the input distribution
    Random.seed!(42)  # For reproducibility
    n_samples = 1000000
    input_samples = rand(Normal(input_mean, input_std), n_samples)

    # Transform through the nonlinear function
    output_samples = inverse_transformed_elu.(input_samples)

    # Create input distribution curve (scaled for visualization)
    y_input = collect(range(input_mean - 4 * input_std, input_mean + 4 * input_std, 100))
    input_pdf = pdf.(Normal(input_mean, input_std), y_input)
    # Scale and position the input PDF (rotated to appear on left side)
    input_scale = 0.15  # Scale factor for visualization
    input_pdf_scaled = input_pdf / maximum(input_pdf) / 3

    # Plot input uncertainty (Gaussian) - rotated to appear on y-axis
    plot!(p, 1.5 .- input_pdf_scaled, y_input,
        fillrange=0.5, fillalpha=0.3, color=:green,
        linewidth=2, label="Input Uncertainty (Gaussian)")

    # Calculate output statistics
    output_mean = mean(output_samples)
    output_std = std(output_samples)

    # Show some sample transformations
    for y_val in range(input_mean - input_std * 3, input_mean + input_std * 3, 11)
        x_val = inverse_transformed_elu(y_val)
        plot!(p, [1.5, x_val], [y_val, y_val], color=:red, alpha=0.3, linewidth=1, label="")
        scatter!(p, [x_val], [y_val], color=:red, alpha=0.6, markersize=2, label="")
    end

    # Create histogram of output values
    nbins = 100
    x_bins = range(inverse_transformed_elu.(max(0.001, input_mean - input_std * 4)), inverse_transformed_elu.(input_mean + input_std * 4), nbins)
    output_hist = fit(Histogram, output_samples, x_bins)
    hist_x = output_hist.edges[1][1:end-1] .+ StatsBase.step(output_hist.edges[1]) / 2
    hist_y = output_hist.weights

    pos = -1.0
    plot!(p, hist_x, hist_y ./ maximum(hist_y) .+ pos,
        fillrange=pos, fillalpha=0.3, color=:orange,
        linewidth=2, label="Ouput Uncertainty (Not gaussian)")
    # Show some sample transformations
    for y_val in range(input_mean - input_std * 3, input_mean + input_std * 3, 11)
        x_val = inverse_transformed_elu(y_val)
        plot!(p, [x_val, x_val], [y_val, pos], color=:red, alpha=0.3, linewidth=1, label="")
        scatter!(p, [x_val], [y_val], color=:red, alpha=0.6, markersize=2, label="")
    end

    plot!(xlim=(1.0, 2.5), ylim=(-1, 2.), grid=false, size=(600, 600))
    savefig("error_propagation_transport.pdf")
    p
end